<a href="https://colab.research.google.com/github/Aggraj/Deep-Learning-CS-6910/blob/main/RBM_Train_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import WandB 

In [2]:
!pip install wandb
import wandb


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from keras.datasets import fashion_mnist
from keras.datasets import mnist


**Read the Fashion MNIST data set, divide the data set as train and validation. There are 10 class in the data set. Initialize all the variables e.g : number of hidden layer size, number of class etc**

In [4]:
# load the Fashion MNIST data set
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_type = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 

Project='ASSIGNMENT 4'


# Split the data set and assign the data values depending on thresholding
X_train = np.array(x_train.reshape(x_train.shape[0], 784,1))         
X_test  = np.array(x_test.reshape(x_test.shape[0], 784,1))           
X_train = (X_train > 126) * 1                                        
X_test  = (X_test > 126) * 1                                         
X_val = X_train[-15000:]                                             
X_train = X_train[0:45000]                                           

Y_train = np.zeros([len(y_train),10,1])
Y_test = np.zeros([len(y_test),10,1])

for i in range(len(y_train)):                                       
  y = np.zeros([10, 1])
  y[y_train[i]] = 1.0
  Y_train[i] = y

for i in range(len(y_test)):                                         
  y = np.zeros([10, 1])
  y[y_test[i]] = 1.0
  Y_test[i] = y                                                      

Y_val = Y_train[-15000:]                                             
Y_train = Y_train[0:45000]        


n_vis = X_train.shape[1]                                         
n_train = X_train.shape[0]                                  
n_val = X_val.shape[0]                                      
n_test = X_test.shape[0]                                    
n_hidden = 64                 
n_class = 10


4431872/4422102 [==============================] - 0s 0us/step


**Define activation functions, Initalize Parameters**

In [5]:

def sigmoid(x) :                                                                              
	
  return 1.0/(1.0+np.exp(-x))

def softmax(x):                                                                               
  return np.exp(x) / np.sum(np.exp(x))  
  
def hidden_r(n,W,image,c):
  return np.random.binomial(1,sigmoid(np.dot(W,image)+c))

def RBM_init_params() :                                                                        
	
  RBM_params = {}
  RBM_params["W"] = np.random.randn(n_hidden, n_vis)*np.sqrt(6./(n_vis + n_hidden))   
  RBM_params["h_b"] = np.zeros((n_hidden,1),dtype=np.float64)                                                     
  RBM_params["v_b"] = np.zeros((n_vis,1),dtype=np.float64)

  return RBM_params

def Class_params_init() :                                                                 

  class_params = {}
  class_params["W"] = np.random.randn(n_class, n_hidden)*np.sqrt(6./(n_class + n_hidden))       
  class_params["b"] = np.zeros((n_class,1),dtype=np.float64)

  return class_params


def hidden_representation(x,parameters) :                                                               
   
    W = parameters["W"]
    h_b = parameters["h_b"]
    hidden_prob = sigmoid(np.dot(W,x)+h_b)
    hidden_rep = np.random.binomial(1,hidden_prob)
    
    return hidden_rep                                   

# **Train the RBM model using Gibbs sampling and Train the classifer**

In [6]:
def Train_RBM(X_train,parameters,k,learning_rate) :                       # Function to train the RBM
	
  W = parameters["W"]
  h_b = parameters["h_b"]
  v_b = parameters["v_b"]
  
  
  for i in range(n_train) :
      v_s = X_train[i]
      v_in   = X_train[i]

      for t in range(k) :                                                               # Markov Chain 
         p_h_g_v = sigmoid(np.dot(W,v_s)+h_b)                                  
         h_s = np.random.binomial(1,p_h_g_v)                                      
         p_v_g_h = sigmoid(np.dot(np.transpose(W),h_s)+v_b)                    
         v_s = np.random.binomial(1,p_v_g_h)                                      # Convert to 0's and 1's using binomial distribution 

      # Update Rule
      W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_in)+h_b),np.transpose(v_in)) - np.dot(sigmoid(np.dot(W,v_s)+h_b),np.transpose(v_s)))
      v_b = v_b + learning_rate*(v_in-v_s)
      h_b = h_b + learning_rate*(sigmoid(np.dot(W,v_in)+h_b) - sigmoid(np.dot(W,v_s)+h_b))

  parameters["W"] = W
  parameters["h_b"] = h_b
  parameters["v_b"] = v_b

  return parameters



In [ ]:
  RBM_params = RBM_init_params()
  epochs = 1
  k = 2
  learning_rate = 0.01

  W      = RBM_params["W"]
  h_b = RBM_params["h_b"]
  v_b = RBM_params["v_b"]

  for j in range(epochs) :
      
      plt.figure(figsize = (10,8))
      subplot_no = 1
          
      for i in range(n_train) :
          v_s    = X_train[i]
          v_in   = X_train[i]

          if ((i%(400) == 0)) and (subplot_no <= 64):

            hidden_reps = np.random.binomial(1,sigmoid(np.dot(W,v_s)+h_b))
            rec_img = np.random.binomial(1,sigmoid(np.dot(np.transpose(W),hidden_reps)+v_b))
            img = np.reshape(rec_img, (28,28))
            plt.subplot(8,8,subplot_no)
            plt.imshow(img, cmap = "gray")
            plt.axis("off")
            subplot_no += 1

          for t in range(k) :                                                                
             p_h_g_v = sigmoid(np.dot(W,v_s)+h_b)                                  
             h_s = np.random.binomial(1,p_h_g_v)                                      
             p_v_g_h = sigmoid(np.dot(np.transpose(W),h_s)+v_b)                    
             v_s = np.random.binomial(1,p_v_g_h)                                      
          # Update Rule
          W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_in)+h_b),np.transpose(v_in)) - np.dot(sigmoid(np.dot(W,v_s)+h_b),np.transpose(v_s)))
          v_b = v_b + learning_rate*(v_in-v_s)
          h_b = h_b + learning_rate*(sigmoid(np.dot(W,v_in)+h_b) - sigmoid(np.dot(W,v_s)+h_b))


      plt.show()

  RBM_params["W"] = W
  RBM_params["h_b"] = h_b
  RBM_params["v_b"] = v_b
